In [106]:
# Importando as bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup

import pandas as pd

import time
from datetime import date

In [107]:
with open('login.txt', 'r') as file:
    linhas = file.readlines()

credentials = {"login": linhas[0].strip(), "password": linhas[1].strip()}


In [108]:
# Criando a instância no WebDriver
srv = Service("chromedriver-linux64/chromedriver") 
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=srv, options=opt)

In [109]:
def login_linkedin(user, password):
    # Abrindo a página de Login do LinkedIn
    driver.get('https://www.linkedin.com/login/')
    time.sleep(1) # aguardar 1 segundos para a página abrir

    # Inserindo o usuário
    usuario = driver.find_element('id','username') # acessando o campo usuário
    usuario.send_keys(credentials['login']) # enviando a string com o usuário

    # Inserindo a senha
    senha = driver.find_element('id','password') # acessando o campo senha
    senha.send_keys(credentials['password']) # enviando a string com a senha

    # Clicando no botão 'Entrar' para acessar a conta no LinkedIn
    driver.find_element("xpath","//button[@type='submit']").click()

login_linkedin(credentials['login'], credentials['password'])

In [110]:
profile_id = "nicolashdx"
url = f'https://www.linkedin.com/in/{profile_id}/'

def get_profile(perfil_url):
    # Acessando o perfil
    driver.get(perfil_url)
    time.sleep(1) # aguardar 1 segundos para a página abrir

def download_pdf():
    try:
        # Clicando no botão '...' na seção de "Introdução" do perfil
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
        time.sleep(1) # aguardar 1 segundo

        # Clicando no botão "Salvar como PDF"
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[2]/div').click()

    except:
        # Clicando no botão 'Mais' na seção de "Introdução" do perfil
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
        time.sleep(1) # aguardar 1 segundo

        # Clicando no botão "Salvar como PDF"
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[2]/div').click()

# Função para ler a página inicial
def read_profile():
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 10 segundos
        if round(fim - inicio) > 10:
            break
    
    # Salvando o código fonte da página em uma variável
    src = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup = BeautifulSoup(src,'lxml')
    
    # Retornando o Código Fonte
    return soup

# Função para Extrair as informações contidas na seção de Introdução da página
def extrair_introducao(codigo_fonte_principal):
    # Obtendo o HTML de extração da introdução
    intro = codigo_fonte_principal.find('div',{'class': 'mt2 relative'})
    
    # Localizando e extraindo o nome do candidato
    nome_loc = intro.find('h1')
    nome = nome_loc.get_text().strip()

    # Localizando e extraindo o título profissional do candidato
    titulo_loc = intro.find('div', {'class': 'text-body-medium break-words'})
    titulo = titulo_loc.get_text().strip()

    # Localizando e extraindo o endereço de residência
    try:
        localizacao_loc = intro.find('span', {'class': 'text-body-small inline t-black--light break-words'})
        localizacao = localizacao_loc.get_text().strip()
    except:
        localizacao = ''

    # Localizando e extraindo a organização de trabalho
    trabalho_loc = intro.find('div', {'class': 'inline-show-more-text'})
    if trabalho_loc == None:
        trabalho = ''
    else:
        trabalho = trabalho_loc.get_text().strip()

    # Localizando e extraindo a instituição da formação acadêmica
    estudo_loc = intro.find_all('div', {'class': 'inline-show-more-text'})
    if estudo_loc == None:
        estudo = ''
    else:
        try:
            estudo = estudo_loc[1].get_text().strip()
        except:
            estudo = ''
        
    # Retornando as informações extraídas
    return nome, titulo, localizacao, trabalho, estudo


# get_profile(url)

# profile = read_profile()

# nome, titulo, localizacao, trabalho, estudo = extrair_introducao(profile)

In [111]:
def extrair_info_formacao_academica(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Separando o Tipo da Formação do Nome do Formação
    tipo_nome_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_nome_formacao = tipo_nome_formacao_loc.get_text().strip().split(',')

    # Tipo da Formação
    tipo_formacao = tipo_nome_formacao[0].strip()

    # Nome da Formação
    nome_formacao = tipo_nome_formacao[1].strip()

    # Extraindo a duração da Formação
    duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    duracao_formacao = duracao_formacao_loc.get_text().strip()
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

def extrair_info_formacao_academica_2(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Extraindo a duração da Formação (caso esteja disponível)
    try:
        duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
        duracao_formacao = duracao_formacao_loc.get_text().strip()
    except:
        duracao_formacao = ''
    
    # Campos não disponíveis `nome_formacao` e `tipo_formacao`
    nome_formacao = ''
    tipo_formacao = ''
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

# Função para extrair as informações referente à Formação Acadêmica mais recente
def extrair_formacao_academica(perfil_id):
    # Acessando a página referente às formações acadêmicas
    formacao_url = 'https://www.linkedin.com/in/{}/details/education/'.format(perfil_id)
    driver.get(formacao_url)
    
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_educacao = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_educacao = BeautifulSoup(src_educacao,'lxml')
    
    # Obtendo o HTML de extração das formações acadêmicas
    educacao = soup_educacao.find('div',{'class':'scaffold-finite-scroll__content'}).find('ul').find_all('li')
            
    # Retornando as informações extraídas
    return educacao

edc_list = extrair_formacao_academica(profile_id)

formacoes = []

for edc in edc_list:
    if(edc.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'})):
        try:
            try:
                nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
            except:
                nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
            formacoes.append({'instituicao': nome_instituicao, 'tipo': tipo_formacao, 'area': nome_formacao, 'duracao': duracao_formacao})
        except:
            nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = '', '', '', ''


In [112]:
formacoes

[{'instituicao': 'Universidade Federal de Minas Gerais',
  'tipo': 'Bacharelado em Engenharia',
  'area': 'Engenharia Elétrica e Eletrônica',
  'duracao': 'mar de 2023 - dez de 2027'},
 {'instituicao': 'Ifes - Instituto Federal do Espírito Santo',
  'tipo': 'Curso Técnico Integrado',
  'area': 'Técnico em Eletrotécnica',
  'duracao': 'fev de 2019 - dez de 2022'}]

In [113]:
'''

time.sleep(10) # aguardar download
driver.close()

'''

'\n\ntime.sleep(10) # aguardar download\ndriver.close()\n\n'